# A/B Test
Generally, A/B tests have two variants, in this case a control and treatment group. We will analyze the converted group to see the effectiveess on treatment. Before going further, here is what the data looks like.

In [1]:
import pandas as pd
import numpy as np
file='/kaggle/input/ab-testing/ab_data.csv'
df=pd.read_csv(file)
df.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


# Brief Analysis
There is an efficient amount of data (more than 100) of balanced data between teatment and control.

In [2]:
df['group'].value_counts()

treatment    147276
control      147202
Name: group, dtype: int64

# Descriptive Stats

In [3]:
df[['group', 'converted']].describe()

,converted
count,294478.000000
mean,0.119659
std,0.324563
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


# Calculate Conversion for Treatment and Control Groups

In [4]:
control = df[df['group'] == 'control'].mean(numeric_only=True).round(4)
control_perc=control['converted']

treatment = df[df['group']=='treatment'].mean(numeric_only=True).round(4)
treatment_perc = treatment['converted']
print(treatment_perc)

0.1189
